### Wann kommt etwas in **fields**, **tags**, **measurements** oder **time** in TinyFlux?

1. **Fields** (Felder):
   - **Fields** enthalten **numerische oder messbare Daten**. Diese Werte ändern sich häufig und werden in der Regel zur Analyse verwendet.
   - Typische Beispiele für **Fields**:
     - Temperaturmessungen (z. B. `"temperature"`)
     - Luftfeuchtigkeit
     - Druck
   - Aus der CSV-Datei: In diesem Fall wird die Spalte **"temperature"** in das `fields`-Dictionary eingefügt.

   **Beispiel im Code**:
   ```python
   fields={
       "temperature": float(row["temperature"])  # Numerische Felder
   }```

2. **Tags** (Schlagwörter):

    - **Tags** sind nicht-numerische Daten, die zusätzliche Informationen über den Datensatz enthalten. Sie sind oft statische Merkmale, die den Kontext eines Datenpunkts beschreiben.
    - Typische Tags:
        - Sensor-ID (z. B. welcher Sensor die Messung vorgenommen hat)
        - Status (z. B. ob der Sensor betriebsbereit oder in Wartung ist)
        - Ort (z. B. der Standort des Sensors)
    - Aus der CSV-Datei: Hier kommen die Spalten "sensor_id" und "status" in das tags-Dictionary.

    **Beispiel im Code**:
   ```python
    tags={
    "sensor_id": row["sensor_id"],  # Nicht-numerische Felder (z.B. Sensor-ID)
    "status": row["status"]  # Status des Sensors (z.B. operational)
    }```

3. **Time** (Zeitstempel):

    - Jeder Datensatz in TinyFlux benötigt einen Zeitstempel, da es sich um eine Zeitserien-Datenbank handelt. Der Zeitstempel gibt an, wann die Messung stattgefunden hat.
    - In der CSV-Datei: Dieser Wert wird in der Spalte "time" gespeichert und muss in ein datetime-Objekt umgewandelt werden, damit TinyFlux den Zeitstempel korrekt verarbeiten kann.

    **Beispiel im Code**:
   ```python
   time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S")  # Zeitstempel wird formatiert
   ```
4. **Measurements** (Messungen):

    **Measurements** bezeichnen, was gemessen wurde. Dies ist eine Kategorisierung für die Art der Messung.
    In deinem Beispiel wurde der Messungstyp als *"blockchain_transactions"* festgelegt, was möglicherweise nicht ideal für Temperaturmessungen ist. Hier könntest du z. B. "temperature_readings" verwenden.

    **Beispiel im Code**:
   ```python
    measurement="temperature_readings"  # Name der Messung
    ```




###  Zusammenfassung:

- Numerische Werte (wie "temperature") kommen in **fields**.
- Nicht-numerische Informationen (wie "sensor_id" und "status") gehen in **tags.**
- Zeitstempel (wie "time") gehen in das **time-Feld** und müssen im richtigen datetime-Format vorliegen.
- Die Art der Messung (wie "temperature_readings") wird als **measurement** festgelegt.

### Was ist ein Point in TinyFlux?

- Ein Point ist der grundlegende Datentyp in TinyFlux, der einen einzelnen Datenpunkt repräsentiert. Jeder Punkt enthält:
    1. time: Den Zeitstempel des Datensatzes (wann wurde die Messung gemacht?).
    2. fields: Die numerischen Messwerte (z. B. Temperatur).
    3. tags: Nicht-numerische Metadaten, die den Kontext des Punktes beschreiben (z. B. Sensor-ID, Status).
    4. measurement: Eine allgemeine Bezeichnung für die Art der Messung (z. B. "temperature_readings").

    Ein Beispiel für die Erstellung eines Point-Objekts:

    ```python
    point = Point(
        time=datetime.now(),  # Aktuelle Zeit
        fields={"temperature": 23.5},  # Messwert für Temperatur
        tags={"sensor_id": "1", "status": "operational"}  # Metadaten
    )

### TinyFlux-Datenbank: Speichern von Daten

- Die TinyFlux-Datenbank speichert Daten als eine Reihe von Zeitpunkten. Jeder neue Datensatz wird als Point eingefügt. Du kannst Daten direkt einfügen, indem du entweder einzelne Punkte (insert()) oder mehrere Punkte auf einmal (insert_multiple()) hinzufügst.

- Beispiel:

    ``` python
    db.insert(point)  # Einzelnen Punkt speichern
    ```

- Der Code in deinem Beispiel liest eine CSV-Datei Zeile für Zeile aus und fügt diese als Point-Objekte in die TinyFlux-Datenbank ein.

### Abfragen in TinyFlux

- TinyFlux bietet flexible Abfragemöglichkeiten, um Daten aus der Datenbank zu extrahieren. Du kannst nach Zeitstempeln, Tags oder Feldwerten filtern.
- Es gibt drei spezielle Query-Typen:
        1. *TimeQuery():* Zum Filtern nach Zeit (z. B. alle Messungen vor oder nach einem bestimmten Zeitpunkt).
        2. *T*agQuery():* Zum Filtern nach Tags (z. B. alle Messungen eines bestimmten Sensors).
        3. *FieldQuery():* Zum Filtern nach numerischen Feldwerten (z. B. alle Messungen, bei denen die Temperatur über 30 Grad lag).
- Beispiel für eine Zeitabfrage:

     ``` python
        time_query = TimeQuery() >= datetime(2023, 1, 1)
        results = db.search(time_query)
    ```
    

### Kombination von Abfragen

- Du kannst verschiedene Abfragetypen kombinieren, um komplexere Suchanfragen zu stellen. Verwende dabei & (AND) oder | (OR), um mehrere Bedingungen zu verknüpfen.

- Beispiel: Alle Messungen nach dem 1. Januar 2024, die den Status "operational" haben und eine Temperatur über 20 Grad aufweisen:
    ``` python

    time_query = TimeQuery() >= datetime(2024, 1, 1)
    tag_query = TagQuery().status == "operational"
    field_query = FieldQuery().temperature > 20.0

    results = db.search(time_query & tag_query & field_query)
    ``` 


### TinyFlux ist In-Memory:

- TinyFlux ist standardmäßig eine in-memory (im Speicher) Datenbank, was bedeutet, dass alle Daten im RAM gespeichert werden. Wenn du die Datenbank mit einer Datei verbindest (wie in deinem Beispiel temperatures.db), werden die Daten auf die Festplatte geschrieben. Dies ermöglicht dir, auch nach einem Neustart des Programms auf die Daten zuzugreifen.

### Zusammenfassung der wichtigen Punkte:

- **Zeitserien-Datenbank:** TinyFlux speichert und verarbeitet Daten, die mit einem Zeitstempel verknüpft sind.
- **Datenstruktur (Point):** Jeder Punkt enthält einen Zeitstempel, Felder (numerische Werte) und Tags (Metadaten).
- **Abfragen:** TinyFlux bietet Query-Typen für Zeit, Tags und Felder. Diese können kombiniert werden, um komplexe Suchanfragen zu erstellen.
- **CSV-Datei:** Der Code liest Daten aus einer CSV-Datei und fügt sie in die TinyFlux-Datenbank ein.
- **Pandas DataFrame:** Die Abfrageergebnisse können in einen DataFrame umgewandelt werden, um die Daten einfacher zu analysieren

Man kann beliebig fields und tags hinzufügen, auch wenn sie nicht im Excel-File sind. Denn:
- Fields: Sind numerische Werte, die du direkt hinzufügen kannst. Wenn ein Wert nicht aus der CSV stammt, könntest du ihn statisch setzen oder berechnen.
- Tags: Sind nicht-numerische Metadaten. Auch hier kannst du beliebige neue Tags definieren und statische oder berechnete Werte zuweisen.

### Beispiel: Hinzufügen von zusätzlichen fields und tags

Nehmen wir an, du möchtest ein zusätzliches field namens "humidity" und ein tag namens "location" hinzufügen. Diese Felder sind nicht in der CSV, aber du könntest ihnen einen festen Wert oder einen Wert auf Basis anderer Logik zuweisen.

``` python
# Read CSV and insert data into TinyFlux
with open("example_data/synthetic_temperature_readings.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            # Hier wird der Zeitstempel zugewiesen
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="temperature_readings",  # Du kannst hier den Messungstyp anpassen

            # Fields: Du kannst beliebig weitere numerische Felder hinzufügen
            fields={
                "temperature": float(row["temperature"]),
                "humidity": 50.0  # Füge eine konstante Luftfeuchtigkeit hinzu, falls nicht in CSV
            },

            # Tags: Du kannst beliebige zusätzliche Metadaten (nicht-numerische) hinzufügen
            tags={
                "sensor_id": row["sensor_id"],
                "status": row["status"],
                "location": "Berlin"  # Füge einen festen Standort hinzu, wenn er nicht in der CSV ist
            }
        )
        # Der erstellte Datenpunkt (point) wird in die TinyFlux-Datenbank eingefügt
        db.insert(point)
``` 
### Was passiert, wenn Felder oder Tags nicht in der CSV vorhanden sind?

- Wenn ein Feld oder Tag, das du hinzufügst, nicht in der CSV vorhanden ist, erhältst du keinen Fehler, solange du dem Feld oder Tag manuell einen Wert zuweist, wie im Beispiel. TinyFlux fügt es einfach mit dem Wert ein, den du zugewiesen hast.

- Fehler vermeiden: Achte darauf, dass alle Felder und Tags, die du im Code anführst, entweder in der CSV existieren oder dass du sicherstellst, dass ihnen im Code ein gültiger Wert zugewiesen wird.

### Zusammenfassung:

Ja, du kannst beliebig viele Felder (fields) und Tags (tags) hinzufügen, selbst wenn sie nicht in der CSV-Datei sind. Du musst ihnen jedoch manuell Werte zuweisen, entweder als feste Werte oder auf Grundlage von Berechnungen. TinyFlux bietet hier viel Flexibilität, um zusätzliche Informationen zu speichern, die möglicherweise nicht direkt aus der CSV kommen.

In [ ]:
# durchschnitt
df_result_field['temperature'].mean()

# SUmme
df_result_field['temperature'].sum()

# Die Methode df_result_field['temperature'].count() gibt die Anzahl der nicht-leeren Werte (nicht-NaN-Werte) in der Spalte 'temperature' des DataFrames df_result_field zurück.
df_result_field['temperature'].count()

# min und max
df_result_field['temperature'].max()
df_result_field['temperature'].min()

# Median
df_result_field['temperature'].median()

# Standardabweichung
df_result_field['temperature'].std()

# komplette statistik
df_result_field['temperature'].describe()
